**This piece of code is part of an on-going project to generate ideas for projects in the field of artificial intelligence**

In this code we gather data of projects titles from several URLs then preprocess the data to be used in text generation model

In [ ]:
# scrape past stanford projects titles
from bs4 import BeautifulSoup
import requests
import re
result = requests.get("http://cs229.stanford.edu/projects.html?fbclid=IwAR1GZEZmRqaESnYsqG1te5MUiCL5mBLe_DdVvJXFQhEBjR3OmmjaFHMF3NA")
c = result.content
soup = BeautifulSoup(c)
pattern = re.compile(r'Previous projects')
linksSec = soup.find('h3', text=pattern).find_next_sibling('div').findAll('a',href=True)
links = []
for link in linksSec[:7]:
  links.append('http://cs229.stanford.edu/'+link['href'])
projects = []
for link in links[:-2]:
  result = requests.get(link)
  c = result.content
  soup = BeautifulSoup(c)
  projects.extend(soup.find_all("p", {"class": "project-title"},text=True))
for link in links[-2:]:
  result = requests.get(link)
  c = result.content
  soup = BeautifulSoup(c)
  projects.extend(soup.find_all("b",text=True))

Save data to ideas.txt file

Notice: this file includes other data gathered from several URLs

In [ ]:
file_object = open('ideas.txt', 'a')
for project in projects:
  file_object.write(project.text + '\n')
file_object.close()

Some text preprocessing

In [ ]:
#!pip3 install contractions
import contractions
def expand_contractions(text):
    text = contractions.fix(text)
    return text

In [ ]:
import io
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# word_tokenize accepts
# a string as an input, not a file.
stop_words = set(stopwords.words('english'))
file1 = open("ideas.txt",'r')
Lines = file1.readlines()
lemmatizer = WordNetLemmatizer()
new_doc = ''
for line in Lines:
  words = line.split()
  new_line = ''
  for r in words:
    if not r in stop_words:
      new_line = new_line + ' ' + lemmatizer.lemmatize(r.lower())
  
  appendFile = open('filteredideas.txt','a')
  appendFile.write(expand_contractions(new_line.strip()+'\n'))
  appendFile.close()

In [ ]:
!pip3 install git+git://github.com/minimaxir/textgenrnn.git

**Using pretrained model**

In [ ]:
from textgenrnn import textgenrnn
textgen = textgenrnn()
textgen.train_from_file('filteredideas.txt', num_epochs=30)
textgen.generate(5)

**for me:
next remove punctuations but try to leave stopwords or at least leave some of them**